In [39]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model, datasets
from sklearn.model_selection import train_test_split
import sklearn
import pandas as pd

In [40]:
df = pd.read_csv("fully_features.csv")

In [41]:
df.shape

(792, 13)

In [42]:
labels = pd.read_csv("all_our_annotations.csv", sep=":::")

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [43]:
labels.head()

,sequence,char1,char2,manualLabel,sid,Sentence
0,1,0,3,p,s0,Many_-1 years_-1 after_-1 king_-1 Agamemnon_3 ...
1,1,0,3,p,s9,"Together_-1 ,_-1 Orestes_0 and_-1 Electra_1 pl..."
2,1,0,3,p,s10,With_-1 the_-1 eager_-1 support_-1 of_-1 the_-...
3,1,0,3,p,s12,"Unexpectedly_-1 ,_-1 Clytamnestra_5 comes_-1 t..."
4,1,0,3,p,s13,He_3 claims_-1 to_-1 be_-1 a_-1 stranger_-1 be...


In [44]:
all_data = df.merge(labels)

In [45]:
all_data.head()
all_data.shape

(792, 17)

In [46]:
all_data = all_data.drop(['Unnamed: 0', 'char1', 'char2', 'Sentence'], axis=1)

In [47]:
X = all_data.drop(['manualLabel', 'sequence', 'sid'], axis=1)
Y = all_data.manualLabel
 
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = .20, random_state = 1234)

In [48]:
logreg = linear_model.LogisticRegression(C=1e5, multi_class = 'ovr')

# we create an instance of Neighbours Classifier and fit the data.
logreg.fit(X_train, y_train)

preds = logreg.predict(X_test)

correct = 0 
total = 0 

for pred, target in zip(preds, y_test):
    if pred == target:
        correct += 1
    total += 1
    
accuracy = correct/total 
print(accuracy)




0.5911949685534591


In [49]:
preds.shape

(159,)

In [50]:
results = pd.DataFrame()
print(preds.shape)
print(y_test.shape)
results['pred'] = preds
results['y_test'] = np.array(y_test)
results['correct'] = 0

(159,)
(159,)


In [51]:
results.head()

,pred,y_test,correct
0,p,p,0
1,p,u,0
2,p,u,0
3,p,p,0
4,p,p,0


In [52]:
results['correct'] = (results['pred'] == results['y_test']).astype(int)

In [53]:
results.head()

,pred,y_test,correct
0,p,p,1
1,p,u,0
2,p,u,0
3,p,p,1
4,p,p,1


In [54]:
results['correct'].sum()/results['correct'].count()

0.5911949685534591

In [55]:
preds

array(['p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p',
       'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p',
       'p', 'u', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p',
       'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'u', 'p',
       'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'u', 'p',
       'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p',
       'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p',
       'p', 'p', 'p', 'u', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p',
       'p', 'p', 'p', 'p', 'p', 'p', 'u', 'p', 'p', 'p', 'p', 'p', 'p',
       'p', 'p', 'p', 'p', 'p', 'n', 'p', 'p', 'p', 'p', 'p', 'p', 'p',
       'p', 'p', 'p', 'u', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p',
       'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p',
       'u', 'p', 'p'], dtype=object)

In [57]:
from sklearn.metrics import average_precision_score

CF = sklearn.metrics.confusion_matrix(y_test.astype(str), preds)

labs = ['n', 'p', 'u']

FP = {}
for r, label in enumerate(labs):
    fp = 0
    for c in range(3):
        if r != c: 
            fp += CF[r][c]
        FP[label] = fp           
print(FP)


FN = {}
for c, label in enumerate(labs):
    fn = 0 
    for r in range(3):
        if c != r: 
            fn += CF[r][c]
        FN[label] = fn
print(FN)

TP = {}
for r, label in enumerate(labs):
    for c in range(3):
        if r == c:
            TP[label] = CF[r][c]

print(TP)

{'n': 29, 'p': 5, 'u': 31}
{'n': 0, 'p': 58, 'u': 7}
{'n': 1, 'p': 93, 'u': 0}


In [59]:
# calculating precision and recall 

# precision = TP / TP + FP 

# recal = TP / TP + FN 

results = {'n': {}, 'p': {},'u': {}}

for lab in labs:
    results[lab]['precision'] = TP[lab] / (TP[lab] + FP[lab])
    results[lab]['recall'] = TP[lab] / (TP[lab] + FN[lab])
    results[lab]['F1'] = 2 * ((results[lab]['precision'] * results[lab]['recall']) / (results[lab]['precision'] + results[lab]['recall']))
    

print(results)
    

{'n': {'precision': 0.03333333333333333, 'recall': 1.0, 'F1': 0.06451612903225806}, 'p': {'precision': 0.9489795918367347, 'recall': 0.6158940397350994, 'F1': 0.746987951807229}, 'u': {'precision': 0.0, 'recall': 0.0, 'F1': nan}}


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':
